In [9]:
# Import relevant functionality
import os
import asyncio
import requests
import urllib.parse
import json
import configparser
from datetime import date
from msgraph.generated.models.o_data_errors.o_data_error import ODataError
from configparser import SectionProxy
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_openai import OpenAI
from langchain_core.runnables import ConfigurableField
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv
from configparser import SectionProxy
from azure.identity import DeviceCodeCredential
from langgraph.types import interrupt
from msgraph_beta import GraphServiceClient
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode
from typing_extensions import TypedDict


# Graph permissions scopes
# For multiple scopes, separate with a space
scopes = 'user.read Calendars.Read Files.Read.All Sites.Read.All'

#load the environment
load_dotenv("C:\\Users\\sakov\\bootcamp\\LangChain Agents\\.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
GRAPH_ACCESS_TOKEN = os.getenv("GRAPH_ACCESS_TOKEN")


@tool
def ragforHR(query: str) -> str:
    """Get the answers to user enquiries related to HR queries like holidays, DTO, payroll, benefits, insurance, salary structure and perks for employees belonging to various regions like Puget Sound or Canada"""
    url = "https://graph.microsoft.com/beta/copilot/retrieval"
    headers = {
        "Authorization": f"Bearer {GRAPH_ACCESS_TOKEN}",
        "Content-Type": "application/json; charset=utf-8"}
    data = {
        "QueryString": query,
        "filterExpression": "(path:\"https://microsoft.sharepoint.com/sites/HRweb/\")",
        "resourceMetadata": ["author"],
        "MaximumNumberOfResults": 4
    }
    response = requests.post(url, headers=headers, json=data)
    #response = requests.post(url, json=data)
    print("Status Code", response.status_code)
    #print("JSON Response ", response.json())
    return response


tools = [ragforHR]

llm = ChatOpenAI(model="gpt-4o")

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant who knows all about Microsoft HR policies. You can also give information to users about the holidays in their regions. Their DTO policies. Their timeoff policies. Anything they want to know about their employee benefits. The access_tokens are available in the tools so don't ask the users again. You can also tell about SAI DEEPTHI KOVVURU a lot. Please answer any questions asked about SAI DEEPTHI KOVVURU WITHOUT ASKING FOR ACCESS TOKEN AS IT IS AVAILABLE TO YOU. You can find the access tokens and the API keys in the environment files."), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

# Define the function that calls the model
async def call_model(state: MessagesState):
    prompt = prompt.invoke(state)
    response = model.ainvoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = ChatMessageHistory(session_id="test-session")
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "def234"}}

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)
await agent_with_chat_history.ainvoke(
    {"input": "Does Microsft require me to get manager approval for DTO?"},
    config={"configurable": {"session_id": "def234"}},
)



> Entering new AgentExecutor chain...
At Microsoft, Discretionary Time Off (DTO) is designed to offer employees flexibility in managing their time away from work. Generally, DTO does not require manager approval in the traditional sense. However, it's important for employees to communicate and coordinate with their managers and teams to ensure that their absence does not disrupt work or impact project deadlines. This means discussing and aligning on the timing and duration of the time off with your manager. Always check the latest internal policies or your employee handbook, as practices can vary slightly by team or region.

> Finished chain.


{'input': 'Does Microsft require me to get manager approval for DTO?',
 'chat_history': [],
 'output': "At Microsoft, Discretionary Time Off (DTO) is designed to offer employees flexibility in managing their time away from work. Generally, DTO does not require manager approval in the traditional sense. However, it's important for employees to communicate and coordinate with their managers and teams to ensure that their absence does not disrupt work or impact project deadlines. This means discussing and aligning on the timing and duration of the time off with your manager. Always check the latest internal policies or your employee handbook, as practices can vary slightly by team or region."}